In [1]:
%%time
import warnings
warnings.filterwarnings('ignore')
from functools import partial
from collections import defaultdict, namedtuple
import numpy as np
import pandas as pd
import scipy
from sklearn.model_selection import KFold
from tqdm.notebook import tqdm
from itertools import product

from video699.screen.semantic_segmentation.fastai_detector import *
from video699.screen.semantic_segmentation.common import *
from video699.screen.semantic_segmentation.postprocessing import *
from video699.screen.semantic_segmentation.evaluation import *

CPU times: user 8.02 s, sys: 1.13 s, total: 9.15 s
Wall time: 6.6 s


In [2]:
resize_factor = [8]
frozen_epochs = [4, 7]
unfrozen_epochs = [4, 7]
base_lower_bound = [5, 7, 10, 15]
erosion_dilation_kernel_size = [20, 50, 80, 150]
ratio_split_lower_bound = [0.3, 0.4, 0.5, 0.7, 0.8, 0.9]

In [3]:
# resize_factor = [8]
# frozen_epochs = [0]
# unfrozen_epochs = [0]
# base_lower_bound = [15]
# erosion_dilation_kernel_size = [20,25]
# ratio_split_lower_bound = [0.7]

In [4]:
train = list(product(resize_factor, frozen_epochs, unfrozen_epochs))
train_names = ['resize_factor', 'frozen_epochs', 'unfrozen_epochs']

post_processing = list(product(base_lower_bound, erosion_dilation_kernel_size, ratio_split_lower_bound))
post_processing_names = ['base_lower_bound', 'erosion_dilation_kernel_size', 'ratio_split_lower_bound']

all_frames = [frame for video in ALL_VIDEOS for frame in video]

detector = FastAIScreenDetector()
actual_detector = AnnotatedSampledVideoScreenDetector()

In [5]:
def filtered_by(name, used):
    return str(name) in [str(frame.pathname) for frame in used] and 'frame' in str(name)

def split_by(name, validation):
    return str(name) in [str(frame.pathname) for frame in validation]

In [6]:
def model_selection(frames, train_names, post_processing_names, default_filtered_by, default_split_by):
    def make_splits(frames):
        Split = namedtuple('Split', ['train', 'valid'])
        kf = KFold(n_splits=5, shuffle=True, random_state=123)
        splits = {}
        for j, split in enumerate(kf.split(frames)):    
            train_frames = [frames[index] for index in split[0]]
            valid_frames = [frames[index] for index in split[1]]
            splits[j] = Split(train=train_frames, valid=valid_frames)
        return splits

    splits = make_splits(frames)
    df_all = pd.DataFrame(columns=train_names + post_processing_names + ['iou', 'wrong_count', 'kfold_split'])

    for train_values in tqdm(train):
        resize_factor, frozen_epochs, unfrozen_epochs = train_values
        CONFIGURATION['resize_factor'] = str(resize_factor)
        CONFIGURATION['frozen_epochs'] = str(frozen_epochs)
        CONFIGURATION['unfrozen_epochs'] = str(unfrozen_epochs)

        for j in splits.keys():
            filtered_by = partial(default_filtered_by, used=splits[j].train + splits[j].valid)
            split_by = partial(default_split_by, validation=splits[j].valid)

            detector = FastAIScreenDetector(filtered_by=filtered_by, valid_func=split_by)
            detector.train()
            
            valid_frames = [frame for frame in all_frames if split_by(frame.pathname)]
            actuals = [actual_detector.detect(frame) for frame in valid_frames]
            sem_preds = detector.semantic_segmentation_batch(valid_frames)

            for post_processing_values in post_processing:    
                preds = detector.post_processing_batch(sem_preds, valid_frames, **dict(zip(post_processing_names, post_processing_values)))
                wrong_count, ious, _ = evaluate(actuals, preds)
                iou_score = np.nanmean(ious)
                wrong_count = len(wrong_count)
                df_all.loc[len(df_all)] = train_values + post_processing_values + (iou_score, wrong_count, j)
    return df_all

In [7]:
def convert_params(best_params):
    converted_params = []
    for i, par in enumerate(best_params):
        if par.is_integer():
            converted_params.append(int(par))
        elif isinstance(par, np.int64) or isinstance(par, np.float64):
            converted_params.append(par.item())
        else:
            converted_params.append(par)
    best_params = tuple(converted_params)
    
    return best_params

### Lecture-wise 5-fold cross validation

In [8]:
# from fastai.vision import *
# paths = get_image_files(detector.videos_path, recurse=True)
# filtered_by()

In [9]:
kf = KFold(n_splits=5, shuffle=True, random_state=123)
df_best_models = pd.DataFrame(columns=train_names + post_processing_names + ['iou', 'wrong_count'])

for i, split in tqdm(enumerate(kf.split(all_frames))):
    print(f"###################### Split No. {i}")
    other_frames = [all_frames[index] for index in split[0]]
    test_frames = [all_frames[index] for index in split[1]]
    
    # Model selection
    df_all = model_selection(other_frames, train_names, post_processing_names, filtered_by, split_by)    
    df_all['wrong_count'] = df_all['wrong_count'].astype(int)
    best_params = df_all.groupby(train_names + post_processing_names).mean().sort_values(by=['wrong_count', 'iou']).iloc[0].name
    best_params = convert_params(best_params)
    best_params = dict(zip(train_names + post_processing_names, best_params))
    test_filtered_by = partial(filtered_by, used=all_frames)
    test_split_by = partial(split_by, validation=test_frames)
    
    
    best_detector = FastAIScreenDetector(filtered_by=test_filtered_by, valid_func=test_split_by)
    best_detector.train(**best_params)
    
    test_frames = [frame for frame in all_frames if test_split_by(frame.pathname)]
    actuals = [actual_detector.detect(frame) for frame in test_frames]
    preds = [best_detector.detect(frame) for frame in test_frames]
    
    wrong_count, ious, _ = evaluate(actuals, preds)
    iou_score = np.nanmean(ious)
    wrong_count = len(wrong_count)
    df_best_models.loc[len(df_best_models)] = tuple(best_params.values()) + (iou_score, wrong_count)
    df_best_models.to_csv('cross_validation_results_image_wise.csv', index=False)
    
df_best_models.to_csv('cross_validation_results_image_wise.csv', index=False)

0it [00:00, ?it/s]
  0%|          | 0/4 [00:00<?, ?it/s]

###################### Split No. 0


epoch,train_loss,valid_loss,acc,dice,iou_sem_seg,time
0,0.388863,0.152626,0.940612,0.874231,0.795262,00:45
1,0.201911,0.057415,0.979826,0.960039,0.928533,00:49
2,0.125444,0.033549,0.987479,0.974957,0.952560,00:51
3,0.091997,0.038960,0.984975,0.971454,0.948013,00:49


epoch,train_loss,valid_loss,acc,dice,iou_sem_seg,time
0,0.046139,0.031090,0.989721,0.979137,0.961982,00:51
1,0.040536,0.021564,0.991793,0.984543,0.970247,00:49
2,0.029451,0.018319,0.993226,0.987354,0.975723,00:49
3,0.024288,0.014892,0.994108,0.989054,0.978675,00:55


epoch,train_loss,valid_loss,acc,dice,iou_sem_seg,time
0,0.207827,0.074476,0.970398,0.933303,0.891105,00:48
1,0.167140,0.043669,0.984561,0.953854,0.927349,00:45
2,0.117859,0.040899,0.984196,0.952868,0.925816,00:45
3,0.076730,0.033948,0.986787,0.973103,0.950082,00:44


...

epoch,train_loss,valid_loss,acc,dice,iou_sem_seg,time
0,0.021563,0.018911,0.992593,0.986047,0.972718,00:51
1,0.027772,0.016594,0.993512,0.987680,0.975775,00:52
2,0.023121,0.012044,0.995151,0.990883,0.982066,00:54
3,0.018044,0.011335,0.995847,0.992353,0.984958,00:54


epoch,train_loss,valid_loss,acc,dice,iou_sem_seg,time
0,0.311320,0.130084,0.941423,0.883120,0.807537,00:52
1,0.174250,0.046208,0.983086,0.951099,0.921896,00:47
2,0.118429,0.050997,0.983474,0.952504,0.925034,00:50
3,0.089279,0.039058,0.986001,0.956566,0.933078,00:51
4,0.060602,0.039509,0.987224,0.962804,0.944241,00:49
5,0.044007,0.033619,0.988924,0.966059,0.950614,00:49
6,0.032417,0.032736,0.989244,0.966915,0.952243,01:10


epoch,train_loss,valid_loss,acc,dice,iou_sem_seg,time
0,0.021317,0.036205,0.988868,0.965256,0.949431,01:06
1,0.024572,0.036029,0.990367,0.980086,0.964653,01:10
2,0.027724,0.030428,0.991660,0.983021,0.969447,00:55


Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/xbankov/anaconda3/envs/video699/lib/python3.7/multiprocessing/queues.py", line 242, in _feed
    send_bytes(obj)
  File "/home/xbankov/anaconda3/envs/video699/lib/python3.7/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/home/xbankov/anaconda3/envs/video699/lib/python3.7/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/home/xbankov/anaconda3/envs/video699/lib/python3.7/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
BrokenPipeError: [Errno 32] Broken pipe
  File "/home/xbankov/anaconda3/envs/video699/lib/python3.7/multiprocessing/queues.py", line 242, in _feed
    send_bytes(obj)
  File "/home/xbankov/anaconda3/envs/video699/lib/python3.7/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/home/xbankov/

KeyboardInterrupt: 